# Identification of population events and core neurons

Here we find first population events, we quantify them, and we extract their core neurons.   
This analysis extends (from step 4.3 on) that performed by [Filipchuk et al. 2022](https://www.nature.com/articles/s41593-022-01168-5):    

1. Compute population binned firing rate (bin)    
1.1 firing rate     
1.2 smooth it, and find the baseline

2. Establish significance threshold for population events    
2.1 compute Inter-Spike Intervals (ISI) of the original spiketrains    
2.2 reshuffle ISI to create (100 or 1000) surrogates    
2.3 compute the population instantaneous firing rate for each surrogate time-binned rasterplot    

3. Find population events    
3.1 instantaneous threshold is the 99% of the surrogate population instantaneous firing rate    
3.2 smoothed firing rate    
3.3 the peaks above intersections of smoothed fr and threshold mark population events    
3.4 the minima before and after a peak are taken as start and end times of the population event    

4. Find clusters of events    
4.1 produce a cell id signature vector of each population event    
4.2 perform clustering linkage by complete cross-correlation of event vectors    
4.3 produce surrogates clusters to establish a cluster significance threshold (varying between 60% and 99%)    
4.4 find the event reproducibility within each cluster (cluster events cross-correlation)     
4.5 check stimulus specificity of reproducible events

5. Find core neurons
5.1 take all neurons participating to a cluster of events    
5.2 use the a percentage (from 60 to 99%) of the cluster event reproducibility as core significance threshold    
5.3 if the occurrence frequency of a neuron is beyond threshold, then the neuron is taken as core    
5.4 remove core neurons if firing unspecifically within and outside their cluster    

All analyses, surrogates, and figures will be saved in the corresponding experiment folder. 

In [ ]:
random.seed(123456)
np.random.seed(123456)

# defaults
plt.rcParams['image.cmap'] = 'viridis' # works in old and newer versions of matplotlib

## 1. Compute population binned firing rate
##### 1.1 Firing rate as in the [Neuronal Dynamics book](https://neuronaldynamics.epfl.ch/online/Ch7.S2.html)

In [ ]:
fr = firinghist(start_time, stop_time, scan_spiketrains, bin_size=frame_duration) #
print("    population firing: {:1.2f}±{:1.2f} sp/frame".format(np.mean(fr),np.std(fr)) )
# print("    frames:",len(fr))

##### 1.2 smoothing the firing rate and finding the baseline

In [ ]:
# Smoothed firing rate: Savitzky-Golay 1D filter
smoothed_fr = savgol_filter(fr, window_length=7, polyorder=3) # as in Filipchuk
smoothed_fr[smoothed_fr<0.] = 0. # filtering can bring the firing fit below zero

# Baseline: Eilers-Boelens 1D filter
baseline_fr = baseline(smoothed_fr, l=10**8, p=0.01) # as in Filipchuk

# print(len(smoothed_fr))
# # print(video_timestamps[exp_istart:].shape) # (53046,)
# # print(len(np.arange(exp_tstart, video_timestamps[-1], 0.025)))
# # print((signal.decimate(video_timestamps[(np.abs(video_timestamps - exp_tstart)).argmin():], 2)).shape)
# print(len(motSVD_1c))
# motSVD_1clong = np.interp(np.linspace(0,len(motSVD_1c[exp_istart:]),len(smoothed_fr)), np.arange(len(motSVD_1c[exp_istart:])), motSVD_1c[exp_istart:])
# print(motSVD_1clong.shape)

plotting firing rate, smoothed, and baseline

In [ ]:
fig = plt.figure()
plt.plot(fr, linewidth=0.5, zorder=1)
plt.plot(smoothed_fr, linewidth=0.5, zorder=1)
plt.plot(baseline_fr, linewidth=0.5, zorder=2)
plt.title("mean rate: %.2f (%.2f) sp/frame" % (fr.mean(), fr.std()) )
plt.ylabel('Firing (sp/frame)')
plt.xlabel('Time (frames)')
fig.savefig(exp_path+"/results/firing_scan%s.svg"%scan_id, transparent=True)
plt.close()
fig.clear()
fig.clf()

## 2. Establish significance threshold for population events

##### 2.1 compute Inter-Spike Intervals (ISI) of the original spiketrains 
and take the firing statistics of each cell (for later).

In [ ]:
spiketrainsISI = []
cells_cv = []
cells_firing_rate = []
for st in scan_spiketrains:
    cells_firing_rate.append( firinghist_one(st, np.arange(start_time, stop_time, frame_duration) ) ) # cell firing rate
    spiketrainsISI.append( np.diff( st ) )
print("    cells firing rate: {:1.2f}±{:1.2f} sp/s".format(np.mean(cells_firing_rate),np.std(cells_firing_rate)) )

##### 2.2 reshuffle ISI to create (100 or 1000) surrogates    
and
##### 2.3 compute the population instantaneous firing rate for each surrogate time-binned rasterplot    
Then we (load, if already done, or) generate a number of surrogate firing rates.    
To respect the statistics of each experiment and population, this is done by taking the actual inter-spike-intervals of each cell and shuffle it. The number of spikes will be the same, the intervals will be the same, but not their order.   

In [ ]:
print("... generating surrogates to establish population event threshold")
if os.path.exists(exp_path+"/results/surrogate_fr_scan%s.npy"%scan_id):
    surrogate_fr = np.load(exp_path+"/results/surrogate_fr_scan%s.npy"%scan_id, allow_pickle=True)
    print("... loaded surrogates")
else:
    print("... entering reshuffling")
    surrogate_fr = []
    # for isur in tqdm(range(100), total=100, desc="creating surrogates"):
    for isur in range(100):
        # build surrogate rasterplot
        surrogate_spiketrains = []
        for isi in spiketrainsISI:
            random.shuffle(isi) # in-place function
            if len(isi): # init of timing of first spike
                isi[0] += start_time
            surrogate_spiketrains.append( np.cumsum(isi) )
        # compute the population instantaneous firing rate for each surrogate timebinned rasterplot
        surrogate_fr.append( firinghist(start_time, stop_time, surrogate_spiketrains, bin_size=frame_duration) )
        # print("    performed reshuffling",isur)
    np.save(exp_path+"/results/surrogate_fr_scan%s.npy"%scan_id, surrogate_fr)

## 3. Find population events    

##### 3.1 instantaneous threshold is the 99% of the surrogate population instantaneous firing rate    

The significance threshold is made by the 99% of the surrogate population instantaneous firing rate, plus the baseline to respect slower fluctuations.    
Either flattening the surrogates or averaging over thier instants (keeping the 0th dimension) leads to similar results.

In [ ]:
event_threshold = np.nanpercentile(np.array(surrogate_fr), q=99) +baseline_fr # flattened (single mean value)
# event_threshold = np.nanpercentile(np.array(surrogate_fr), q=99, axis=0) +baseline_fr # keep dimension 0 (100 surrogates)
print("    event size threshold (mean):",np.mean(event_threshold))
del surrogate_fr
%reset_selective -f surrogate_fr

##### 3.2 smoothed firing rate    

The peaks (maximal extrema), above threshold, are the peaks population events.

In [ ]:
print("... find peaks")
peaks = []
fr_mean = np.mean(smoothed_fr)
for peak in signal.argrelextrema(smoothed_fr, np.greater)[0]:
    if smoothed_fr[peak] < event_threshold[peak]:
        continue # ignore peaks below mean
    peaks.append(peak)

##### 3.3 the peaks above intersections of smoothed fr and threshold mark population events    
##### 3.4 the minima before and after a peak are taken as start and end times of the population event    

The minimal extrema (above or below threshold) are possible limits (beginnigs and ends) of population events.

In [ ]:
print("... find minima")
minima = signal.argrelextrema(smoothed_fr, np.less, order=2)[0]
# there can be periods when the firing rate is 0, to be considered minima as well
zerominima = np.where(smoothed_fr == 0)[0]
minima = np.concatenate((minima,zerominima))
minima.sort(kind='mergesort')

The minimum before and after a peak are taken as start and end times of a putative population event.    
The dict list `events` will store beginning, and end (and stimulus, if applicable) of each population event.

In [ ]:
print("... find population events")
events = []
# each peak (beyond threshold) is part of an event
for peak in peaks:
    event = {'start':0, 'end':0, 'start_frame':0, 'end_frame':0, 'stimulus':None, 'color':'gray'} # init
    # the minima (either below or above threshold) before and after the peak are the real limits of the event
    # index of the minimum before the peak
    minimum_pre = minima[ np.argwhere(minima<peak)[-1][0] if len(np.argwhere(minima<peak))>0 else 0 ]
    # start
    event['start'] = minimum_pre
    event['start_frame'] = minimum_pre
    # end
    # index of the minimum after the peak
    mininum_post = minima[ np.argwhere(minima>peak)[0][0] if len(np.argwhere(minima>peak))>0 else 0 ]
    event['end'] = mininum_post
    event['end_frame'] = mininum_post
    if event['start']>=0 and event['end']>event['start']:
        events.append(event)

##### Processing the behavioral indicator
The data from Stringer et al. 2019 also contains face motion energy components. The first principal component captures motion everywhere on the face and strongly correlates with arousal measures (pupil, whisking), see their Fig. 2 and S4.    
Here, we use this component to study the temporal order of population events with respect to motion energy changes.

In [ ]:
if 'video_timestamps' in locals():
    print("... processing behavioral data")
    smoothed_beh = np.ones(len(smoothed_fr)) * -np.inf
    for ievent,event in enumerate(events):
        event_start_time = exp_tstart + event['start'] * frame_duration
        event_end_time = exp_tstart + event['end'] * frame_duration
        # to have a measure of behavior, we take the mean of the first component value of each frame during the event
        meanbeh1c = np.mean(motSVD_1c[np.where((video_timestamps>=event_start_time)&(video_timestamps<=event_end_time))]) / 1000. # A.U. just a factor to ease plotting
        events[ievent]['stimulus'] = meanbeh1c -2. # A.U. to be all negative
        smoothed_beh[event['start']] = meanbeh1c -2. # A.U. to be all negative
        
    # pre-processing the motSVD PC1
    smoothed_beh[smoothed_beh==-np.inf] = np.max(smoothed_beh) # default
    # smooth motion energy curve
    smoothed_beh = savgol_filter(smoothed_beh, window_length=3, polyorder=2)    
    # Find times of changes in motion energy
    smoothed_diff = np.diff(smoothed_beh)
    smoothed_beh_indices = list(np.where(smoothed_diff < np.percentile(smoothed_diff, q=1))[0])
    
    # Count the number of events before and after the change.
    # cycle over series of intervals, to test dependence on interval
    pre_beh_events = [[] for i in range(6)] # intervals event idx
    post_beh_events = [[] for i in range(6)]
    Npre_beh_events = [0 for i in range(6)] # intervals event count
    Npost_beh_events = [0 for i in range(6)] 
    beh_intervals = [0.050, 0.100, 0.150, 0.200, 0.250, 0.300]
    for iint,interval in enumerate(beh_intervals):
        print("    interval", interval)
        for sni in smoothed_beh_indices:
            snitime = exp_tstart + sni * frame_duration
            snitime_pre = snitime - interval # s
            snitime_post = snitime + interval # s
            for ievent,event in enumerate(events):
                event_start_time = exp_tstart + event['start'] * frame_duration
                if snitime_pre < event_start_time and event_start_time < snitime:
                    pre_beh_events[iint].append(ievent)
                    Npre_beh_events[iint] += 1
                if snitime < event_start_time and event_start_time < snitime_post:
                    post_beh_events[iint].append(ievent)
                    Npost_beh_events[iint] += 1
        print("    pre %.3f (%d/%d)"%(Npre_beh_events[iint]/len(events), Npre_beh_events[iint], len(events)) )
        print("    post %.3f (%d/%d)"%(Npost_beh_events[iint]/len(events), Npost_beh_events[iint], len(events)) )
    Npre_beh_events = np.array(Npre_beh_events)/len(events)
    Npost_beh_events = np.array(Npost_beh_events)/len(events)
    # summary
    fig, ax = plt.subplots(figsize=(5,5))
    plt.plot(Npre_beh_events, linewidth=2.5, color='red', zorder=5)
    plt.plot(Npost_beh_events, linewidth=2.5, color='blue', zorder=5)
    plt.ylabel('event occurrences ratio')
    plt.xlabel('interval (sec)')
    plt.ylim([0,1])
    plt.xticks(range(6),beh_intervals)
    fig.savefig(exp_path+"/results/eventsXmovements_scan%s.png"%scan_id, transparent=True, dpi=900)
    plt.close()
    fig.clear()
    fig.clf()

#### plot everything so far
original, threshold, maxima and minima, events

In [ ]:
fig, ax = plt.subplots(figsize=(20,5))
x = np.arange(0,len(smoothed_fr))

for event in events:
    ax.axvspan(event['start'], event['end'], alpha=0.1, color='green', zorder=1)
    
    if 'video_timestamps' in locals():
        ax.plot(event['start'], event['stimulus'], 'co', markersize=1, markeredgewidth=0., zorder=6)
        
if 'video_timestamps' in locals():
    plt.plot(smoothed_beh, linewidth=0.5, color='b', zorder=5)
    for sni in smoothed_beh_indices:
        ax.plot(sni, -1.5, 'yo', markersize=1, markeredgewidth=0., zorder=6)

if 'orientation_interval_frames' in locals():
    for stimframes in orientation_interval_frames[scan_id]:
        for timeframes in stimframes:
            ax.axvspan(timeframes[0], timeframes[1], alpha=0.1, linewidth=0, facecolor='grey', zorder=1)
            
ax.plot(smoothed_fr,linewidth=0.5, color='k', zorder=2)
ax.plot(event_threshold, linewidth=0.5, color='magenta', zorder=5)
plt.plot(baseline_fr, linewidth=0.5, color='orange', zorder=5)
plt.plot(baseline_fr, linewidth=0.5, color='orange', zorder=5)
ax.plot(x[minima], smoothed_fr[minima], 'wo', markersize=1, markeredgewidth=0., zorder=4)
ax.plot(x[peaks], smoothed_fr[peaks], 'rs', markersize=.5, zorder=4)
fig.savefig(exp_path+"/results/Events_population_firing_scan%s.png"%scan_id, transparent=True, dpi=900)
plt.close()
fig.clear()
fig.clf()

## 4. Find clusters of events    

with not enough events, no point in going further ...

In [ ]:
if len(events)<4:
    print("... not enough events to continue analysis (<4)")
#     raise StopExecution
#     # exit()

##### 4.1 produce a cell id signature vector of each population event   
We store both the ids and indexes of all cells firing during the population event

In [ ]:
print("... signatures of population events")
# get the cell ids for each event
events_signatures = [] # N x M, N events and all M idx for each event (0s and 1s)
events_spectrums = [] # N x Z, N events and only Z cids for each event
events_vectors = [] # event vectors for attractor analysis
toBremoved = []

if 'exp_tstart' not in vars() or 'exp_tstart' not in globals():
    exp_tstart = 0

for event in events:
    # signature
    signature = [0 for i in range(len(scan_spiketrains))] # init
    spectrum = []
    evector = {"color":"gray", "variables":np.zeros(len(ophys_cell_ids))}
    # start end of event index are converted to ms in order to search cell ids being active in that window
    tstart = exp_tstart + event['start'] * frame_duration
    tend = exp_tstart + event['end'] * frame_duration
    for idx,spiketrain in enumerate(scan_spiketrains):
        # take the idx if there are spikes within event start and end
        spiketrain = np.array(spiketrain)
        # choose idx based on event limits
        if np.any(np.logical_and(spiketrain>=tstart, spiketrain<tend)):
            spectrum.append( ophys_cell_ids[idx] ) # to store the actual cid and not just the index
            signature[idx] = 1 #
        evector["variables"][idx] = np.mean(cells_firing_rate[idx][event['start']:event['end']])
    # check that the event signature has more than 1 cell
    if np.count_nonzero(spectrum)>1:
        events_spectrums.append( spectrum )
        events_signatures.append( signature )
        events_vectors.append(evector)
    else:
        toBremoved.append(events.index(event))

# removing events with just one cell
for index in sorted(toBremoved, reverse=True):
    del events[index]

events_signatures = np.array(events_signatures)
events_spectrums = np.array(events_spectrums, dtype=object)

#### Some basic population event statistics
Number of events per second, event duration, and size.

In [ ]:
print("    number of events:",len(events))
events_sec = len(events_signatures)/stop_time
print("    number of events per sec:",events_sec)

# events durations and intervals
events_durations = []
events_intervals = []
last_event = None
for event in events:
    events_durations.append(event['end']-event['start'])
    if last_event: # only from the second event on
        events_intervals.append(event['start']-last_event['end'])
    last_event = event

# Population events mean+std Duration
events_durations_f = np.array(events_durations, dtype=float)
events_durations_f = events_durations_f*frame_duration
np.save(exp_path+"/results/events_durations%s.npy"%scan_id, events_durations_f)
print("    events duration: %.3f±%.3f" % (np.median(events_durations_f), np.std(events_durations_f)))
fig = plt.figure()
plt.yscale('log')
hn,hx,_ = plt.hist(events_durations_f, bins='auto')
lims = plt.ylim()
plt.vlines([np.median(events_durations_f)], ymin=lims[0], ymax=lims[1], linestyles='dashed', colors='k')
plt.title("median events duration: %.3f±%.3f" % (np.median(events_durations_f), np.std(events_durations_f)) )
plt.ylabel('event occurrences')
plt.xlabel('event duration (sec)')
plt.yscale('log')
plt.xscale('log')
fig.savefig(exp_path+"/results/Events_durations_scan%s.svg"%scan_id, transparent=True)
plt.close()
fig.clear()
fig.clf()

# Population events size (number of cells per event)
events_size = []
for esignature in events_signatures:
    events_size.append(len(np.nonzero(esignature)[0]))
np.save(exp_path+"/results/events_size%s.npy"%scan_id, events_size)
print("    events size: %.3f±%.3f" % (np.median(events_size), np.std(events_size)))
fig = plt.figure()
hn,hx,_ = plt.hist(events_size, bins='auto')
lims = plt.ylim()
plt.vlines([np.median(events_size)], ymin=lims[0], ymax=lims[1], linestyles='dashed', colors='k')
plt.title("median size: %.3f (%.3f)" % (np.median(events_size), np.std(events_size)) )
plt.ylabel('occurrences')
plt.xlabel('number of cells per event')
plt.yscale('log')
plt.xscale('log')
fig.savefig(exp_path+"/results/Events_size_scan%s.svg"%scan_id, transparent=True)
plt.close()
fig.clear()
fig.clf()

#### 4.2 perform clustering linkage by complete cross-correlation of event vectors  

We compute the Pearson's correlation matrix over events signatures (the patterns of firing cells).

In [ ]:
print("... Similarity of events matrix")
SimilarityMap = np.corrcoef(events_signatures)
fig = plt.figure()
plt.pcolormesh(SimilarityMap)
cbar = plt.colorbar()
fig.savefig(exp_path+"/results/Events_CorrMatrix%s.png"%scan_id, dpi=600, transparent=True)
plt.close()
fig.clear()
fig.clf()

Then we perform the clustering linkage by complete cross-correlation of event signatures.    
The cut-off is arbitrary, but different cut-offs lead to similar results.

In [ ]:
print("... clustering")
print("    linkage")
Z = linkage(events_signatures, method='complete', metric='correlation') #
Z[ Z<0 ] = 0 # for very low correlations, negative values can result
cut_off = 0.7*max(Z[:,2]) # generic cutoff as in matlab

#### 4.3 produce surrogates clusters to establish a cluster significance threshold    
The threshold for cluster significance is established by generating 100 surrogate events_signatures, choosing at random signatures form the population of cells.   
Then we correlate and cluster as above... there will be clusters, happening just by chance due to the finite number of cells, but their internal correlation should not be high.    

In [ ]:
print("    surrogate events signatures for clustering threshold")
if os.path.exists(exp_path+"/results/surrogate_reproducibility_list%s.npy"%scan_id):
    surrogate_reproducibility_list = np.load(exp_path+"/results/surrogate_reproducibility_list%s.npy"%scan_id, allow_pickle=True)
    print("... loaded surrogates")
else:
    print("... entering creation of surrogate signatures")    
    surrogate_reproducibility_list = []
    for csur in range(100):
        surrogate_events_signatures = []
        for evsig in events_signatures:
            surrogate_signature = np.array([0 for i in ophys_cell_indexes]) # init
            surrogate_signature[ np.random.choice(ophys_cell_indexes, size=np.count_nonzero(evsig), replace=False) ] = 1
            surrogate_events_signatures.append(surrogate_signature.tolist())
        # similarity
        surrogate_similaritymap = np.corrcoef(surrogate_events_signatures)
        # clustering
        surrogate_Z = linkage(surrogate_events_signatures, method='complete', metric='correlation') #
        surrogate_Z[ surrogate_Z<0 ] = 0 # for very low correlations, negative values can result
        surrogate_events_assignments = fcluster(surrogate_Z, t=cut_off, criterion='distance')
        # sorting by cluster based on the first element of zip
        surrogate_permutation = [x for _, x in sorted(zip(surrogate_events_assignments, range(len(surrogate_events_signatures))))]
        clustered_surrogate_similaritymap = surrogate_similaritymap[surrogate_permutation] # x
        clustered_surrogate_similaritymap = clustered_surrogate_similaritymap[:,surrogate_permutation] # y
        # cluster reproducibility
        surrogate_events_cluster_sequence = sorted(surrogate_events_assignments) # [ 1 1 1 1 2 2 3 3 3 3 3 ...]
        surrogate_cnt = collections.Counter()
        for word in surrogate_events_cluster_sequence:
            surrogate_cnt[word] += 1
        surrogate_events_cluster_chunks = list(surrogate_cnt.values())
        starti = 0
        for iblock,nblock in enumerate(surrogate_events_cluster_chunks):
            endi = starti+nblock
            # get sub-array
            surrogate_cluster_subarray = np.array(clustered_surrogate_similaritymap[ starti:endi-1, starti:endi-1 ])
            if surrogate_cluster_subarray.size:
                np.fill_diagonal(surrogate_cluster_subarray, 0.0)
                # compute the subarray average
                surrogate_reproducibility_list.append( np.nanmean(surrogate_cluster_subarray) )
            starti = endi
    np.save(exp_path+"/results/surrogate_reproducibility_list%s.npy"%scan_id, surrogate_reproducibility_list)
    
    del surrogate_events_signatures
    %reset_selective -f surrogate_events_signatures

The 95% (or more) correlation of this random cluster will be the threshold for a cluster to be significantly correlated.     
A minimum of 2 events is required for the size of a cluster.

In [ ]:
cluster_reproducibility_threshold = np.percentile(np.array(surrogate_reproducibility_list), 5)
print("    cluster reproducibility threshold:",cluster_reproducibility_threshold)
# number of events in a cluster, even small clusters as long as they pass the reproducibility threshold
cluster_size_threshold = 2 # minimum requirement
print("    cluster size threshold:",cluster_size_threshold)

#### 4.4 find the event reproducibility within each cluster (cluster events cross-correlation)    

The events are rearranged in the correlation map to put together those that are similar.     
The clusters are identified by a (randomly chosen) color that will be kept for the following analyses.    
Gray is the color of clusters below reproducibility threshold.

In [ ]:
# clusters
events_assignments = fcluster(Z, t=cut_off, criterion='distance')
# print(events_assignments)
# print(len(events_assignments))
# ex. [4 4 4 4 2 2 34 4 7 7 7 7 5 5 5 5 4 ... ]

# count clusters
nevents_clusters = np.unique(events_assignments, return_counts=True)[1]
# print("    events/clusters:", nevents_clusters)
# ex. [ 39  19  11  70  15  74  45  13  10  45 ...]
nclusters = len(nevents_clusters)
print("    Total number of clusters:",nclusters)

# color map of the clustered events
cmap = mpcm.get_cmap('rainbow')
cluster_color_array = [mpl.colors.rgb2hex(rgba) for rgba in cmap(np.linspace(0.0, 1.0, nclusters))]
random.shuffle(cluster_color_array) # to have different nearby colors
cluster_color_array = np.array(cluster_color_array)
# print("cluster colors:",len(cluster_color_array))

threshold_map = nevents_clusters < cluster_size_threshold
# print("    removing below size threshold clusters:", np.count_nonzero(threshold_map))
cluster_color_array[threshold_map] = 'gray' # or 'none'

color_array = []
for cluidx in events_assignments:
    color_array.append(cluster_color_array[cluidx-1]) # fcluster returns numeric labels from 0 to nclusters-1
color_array = np.array(color_array)
# ex. color_array = ['#304030', '#008c85', '#008c85', '#005955', '#2db3ac', ...]

# sorting the index of events_signatures based on events_assignments (the first element of zip)
permutation = [x for _,x in sorted(zip(events_assignments, range(len(events_signatures))))]
# permuting all arrays used after
clustered_signatures = events_signatures[permutation]
clustered_spectrums = events_spectrums[permutation]
clustered_event_colors = color_array[permutation]
events = np.array(events)
clustered_events = events[permutation]
np.save(exp_path+"/results/clustered_signatures%s.npy"%scan_id, clustered_signatures)

# reordering SimilarityMap
clustered_SimilarityMap = SimilarityMap[permutation] # x
clustered_SimilarityMap = clustered_SimilarityMap[:,permutation] # y

# Pattern reproducibility - Cluster self-similarity
remove_colors = [cola for cola in color_array]
reproducibility_list = [ 0. for elc in cluster_color_array ]
core_reproducibility = {elc:1. for elc in cluster_color_array}
events_cluster_sequence = sorted(events_assignments) # [ 1 1 1 1 2 2 3 3 3 3 3 ...]
# subarray iteration
ec_cnt = collections.Counter()
for word in events_cluster_sequence:
    ec_cnt[word] += 1
events_cluster_chunks = list(ec_cnt.values())
starti = 0
for iblock,nblock in enumerate(events_cluster_chunks):
    endi = starti+nblock
    cluster_subarray = np.array(clustered_SimilarityMap[ starti:endi-1, starti:endi-1 ])
    if cluster_subarray.size:
        np.fill_diagonal(cluster_subarray, 0.0)
        # compute the subarray average
        reproducibility_list[iblock] = np.nanmean(cluster_subarray)
        # overwrite color
        if np.nanmean(cluster_subarray) < cluster_reproducibility_threshold:
            cluster_color_array[iblock] = "gray"
            clustered_event_colors[starti:endi] = "gray"
        else:
            # Stimulus-free method to detect core neurons:
            # within each cluster of events,
            # cores are those participating to more than 99% of cluster events
            core_reproducibility[cluster_color_array[iblock]] = np.percentile(cluster_subarray, core_reproducibility_perc)
    starti = endi
print("    # clusters (after removing those below reproducibility threshold):", len(cluster_color_array) - collections.Counter(cluster_color_array)['gray'])

# Remove colors to be gray-ed in the event color assignement
events_color_assignments = [col if col in cluster_color_array else "gray" for col in color_array]
# print(events_color_assignments)
for evtidx,_ in enumerate(events):
    events[evtidx]['color'] = events_color_assignments[evtidx]

# save total clusters with their events
clustercolors,eventcounts = np.unique(events_color_assignments, return_counts=True) 
if 'orientation_interval_frames' in locals():
    global_tot_clusters[scan_id].append( clustercolors.tolist() )
    global_tot_clusters[scan_id].append( eventcounts.tolist() )

# plot all
fig, ax = plt.subplots()
plt.pcolormesh(clustered_SimilarityMap)
# loop over cluster colors
clcoord = 0
for csize,ccolor,reproval in zip(nevents_clusters,cluster_color_array,reproducibility_list):
    if ccolor!="gray":
        rect = patches.Rectangle((clcoord, clcoord), csize, csize, linewidth=1., edgecolor=ccolor, facecolor='none')
        ax.add_patch(rect)
        # ax.text(clcoord+1, clcoord+1, "{:1.2f}".format(reproval), color=ccolor, fontsize=2)
    clcoord += csize
cbar = plt.colorbar()
cbar.outline.set_visible(False)
cbar.set_ticks([]) # remove all ticks
for spine in plt.gca().spines.values(): # get rid of the frame
    spine.set_visible(False)
plt.xticks([]) # remove all ticks
plt.tick_params(top=False, bottom=False, left=False, right=False, labelleft=False, labelbottom=False)
fig.savefig(exp_path+"/results/Events_CorrClustered_scan%s.png"%scan_id, transparent=True, dpi=600)
plt.close()
fig.clf()

# Pattern reproducibility by Cluster
fig = plt.figure()
for repri, (reprv, reprc) in enumerate(zip(reproducibility_list,cluster_color_array)):
    plt.bar(repri, reprv, 1., facecolor=reprc)
plt.title('Pattern reproducibility')
plt.ylabel('Auto-correlation')
plt.xlabel('Clusters')
fig.savefig(exp_path+"/results/Pattern_reproducibility_scan%s.png"%scan_id, transparent=True, dpi=600)
plt.close()
fig.clear()
fig.clf()

# Self-similarity index
fig = plt.figure()
hn,hx,_ = plt.hist(reproducibility_list, bins='auto')
bin_centers = 0.5*(hx[1:]+hx[:-1])
plt.plot(bin_centers,hn) ## using bin_centers rather than edges
lims = plt.ylim()
plt.vlines([np.mean(reproducibility_list)], ymin=lims[0], ymax=lims[1], linestyles='dashed', colors='k')
plt.title("Self-Similarity Index: %.3f" % (np.mean(reproducibility_list)))
plt.xlabel('Auto-correlation')
plt.ylabel('Cluster occurrences')
fig.savefig(exp_path+'/results/Pattern_reproducibility_SelfSimilarityIndex%s.png'%scan_id, transparent=True, dpi=600)
fig.savefig(exp_path+'/results/Pattern_reproducibility_SelfSimilarityIndex%s.svg'%scan_id, transparent=True)
plt.close()
fig.clear()
fig.clf()

#### 4.5 Check stimulus specificity of reproducible events

Replot spiketrains, coloring stimuli by orientation and events by cluster.     
There are 16 orientations. Let's take 16 colors and color events by their (reproducible) cluster color.    

In [ ]:
if 'orientation_interval_frames' in locals():
    cmap = plt.get_cmap("tab20")
    orientation_colors = cmap(np.arange(16))
    # print(orientation_colors)

    fig, ax = plt.subplots(figsize=(20,5))

    for oridx,stimframes in enumerate(orientation_interval_frames[scan_id]):
        for timeframes in stimframes:
            ax.axvspan(timeframes[0], timeframes[1], alpha=0.4, linewidth=0, facecolor=orientation_colors[oridx], zorder=1)

    for evtidx,event in enumerate(events):
        ax.axvspan(event['start'], event['end'], alpha=0.4, color=event['color'], zorder=1)

    ax.plot(smoothed_fr,linewidth=0.5, color='k', zorder=2)
    ax.plot(event_threshold, linewidth=0.5, color='magenta', zorder=3)
    plt.plot(baseline_fr, linewidth=0.5, color='orange', zorder=3)
    x = np.arange(0,len(smoothed_fr))
    ax.plot(x[minima], smoothed_fr[minima], 'wo', markersize=1, markeredgewidth=0., zorder=4)
    ax.plot(x[peaks], smoothed_fr[peaks], 'rs', markersize=.5, zorder=4)
    fig.savefig(exp_path+"/results/Events_population_firing_scan%s.svg"%scan_id, transparent=True)
    plt.close()
    fig.clear()
    fig.clf()

We need to check whether reproducible patterns are stimulus-orientation specific.   

To establish a significance level, we want to know what is the chance of random events to be stimulus specific.     
We create surrogate events (leaving the same cluster color) by jittering their timing. 

For each orientation (16), we store the color of the event that follows.    
An event is considered to be triggered by a stimulus if it starts after presentation and no later than the 10 frames (~640ms) after presentation (a third of the inter-stimulus interval).

In [ ]:
if 'orientation_interval_frames' in locals():
    orientation_following_colors = [[] for i in range(16)]
    surrogate_orientation_following_colors = [[] for i in range(16)]

    for oridx,stimframes in enumerate(orientation_interval_frames[scan_id]):
        for stimidx,sframes in enumerate(stimframes): # all but last

            for event in events:
                if event['start'] >= sframes[0] and event['start']<=sframes[0]+10:
                    if event['color'] != "gray":
                        orientation_following_colors[oridx].append(event['color'])

        for isur in range(100):
            surrogate_events = copy.deepcopy(events) 
            for evtidx,sampleevt in enumerate(surrogate_events):
                surrogate_events[evtidx]['start'] = sampleevt['start'] + random.randint(-15, 15)
                surrogate_events[evtidx]['end'] = sampleevt['end'] + random.randint(-15, 15)
            for event in surrogate_events:
                    if event['start'] >= sframes[0] and event['start']<=sframes[0]+10:
                        if event['color'] != "gray":
                            surrogate_orientation_following_colors[oridx].append(event['color'])

We also store, for each orientation, which and how many reproducible patterns (clusters) happened.

In [ ]:
if 'orientation_interval_frames' in locals():
    for oridx,followers in enumerate(orientation_following_colors):
        # print(oridx)
        # print(np.unique(followers, return_counts=True))
        stimclusters,stimcounts = np.unique(followers, return_counts=True) # (array(['#80ffb4'], dtype='<U7'), array([2]))
        if len(stimclusters):
            for stcl,stcnt in zip(stimclusters,stimcounts):
                if stcl in global_cluster_tuning[oridx].keys():
                    global_cluster_tuning[oridx]["%s,%s"%(stcl,scan_id)] += stcnt
                else:
                    global_cluster_tuning[oridx]["%s,%s"%(stcl,scan_id)] = stcnt

    for oridx,followers in enumerate(surrogate_orientation_following_colors):
        # print(oridx)
        # print(np.unique(followers, return_counts=True))
        stimclusters,stimcounts = np.unique(followers, return_counts=True) # (array(['#80ffb4'], dtype='<U7'), array([2]))
        if len(stimclusters):
            for stcl,stcnt in zip(stimclusters,stimcounts):
                if stcl in global_surrogate_cluster_tuning[oridx].keys():
                    global_surrogate_cluster_tuning[oridx]["%s,%s"%(stcl,scan_id)] += stcnt
                else:
                    global_surrogate_cluster_tuning[oridx]["%s,%s"%(stcl,scan_id)] = stcnt

We also store the cell ids of events by cluster color for later comparison to orientation tuning:

In [ ]:
scan_clustered_spectrums = {}
for cluster_evtcol,cluster_spectrum in zip(clustered_event_colors, clustered_spectrums):
    if "%s,%s"%(cluster_evtcol,scan_id) in scan_clustered_spectrums.keys():
        scan_clustered_spectrums["%s,%s"%(cluster_evtcol,scan_id)].append( cluster_spectrum )
    else:
        scan_clustered_spectrums["%s,%s"%(cluster_evtcol,scan_id)] = []
        scan_clustered_spectrums["%s,%s"%(cluster_evtcol,scan_id)].append( cluster_spectrum )

### 5. Find core neurons
#### 5.1 take all neurons participating to a cluster of events    
For each cluster above significance threshold, we count how many times each cell participated to the cluster's events.

#### 5.2 use the a percentage of the cluster event reproducibility as core significance threshold    
Cores identification is by frequence of occurrence (arbitrarily imposed from the outside, from 60% to 99%).

#### 5.3 if the occurrence frequency of a neuron is beyond threshold, then the neuron is taken as core    

In [ ]:
print("... finding cluster cores")
clusters_cores = []
clusters_cores_by_color = {ecolor:list() for ecolor in clustered_event_colors}
cluster_color_cores = [[] for clsp in clustered_spectrums]
currentcl = clustered_event_colors[0]
cluster_events_list = []
for cl_idlist, cl_color in zip(clustered_spectrums, clustered_event_colors):
    # when the color changes, plot the map and reset containers
    if currentcl != cl_color:
        # find common subset of cells in a clusters
        cid_counter = {}
        for event_cids in cluster_events_list:
            for cidc in event_cids:
                if cidc in cid_counter:
                    cid_counter[cidc] += 1/len(cluster_events_list)
                else: # create
                    cid_counter[cidc] = 1/len(cluster_events_list)
        cluster_core = []
        for cidkey,cidprob in cid_counter.items():
            # Cores identification, independent of stimuli
            # A cell is considered 'core' of multiple events if it has
            # a frequence of occurrence > core_reproducibility (%)
            if currentcl in core_reproducibility and cidprob >= core_reproducibility[currentcl]:
                cluster_core.append(cidkey)
                clusters_cores_by_color[currentcl].append(cidkey)
        if len(cluster_core)>0:
            clusters_cores.append(cluster_core)
        # reset containers
        cluster_events_list = []
        currentcl = cl_color
    # while the color is the same, append the idx to the current ensemble
    cluster_events_list.append( cl_idlist )

#### 5.4 remove core neurons if firing unspecifically within and outside their cluster    
We want to check whether a core is firing unspecifically inside and outside of events.    
For each cluster color, take all its events and group or mask the intervals from the rest of the times.    
Then for each core of this cluster, avg rate inside events - avg outside events.   

In [ ]:
print("    removing cores firing unspecifically")

np_cells_firing_rate = np.array(cells_firing_rate)
for caidx,ecolor in enumerate(cluster_color_array):
    if ecolor=='gray':
        continue
    # prepare a row_mask to contain all False for events intervals.
    # It will be used to compute the inside-cluster-events firing rate
    # Its inverse will be used to compute the outsite-cluster-events firing rate
    row_mask = np.array([False for el in range(np_cells_firing_rate.shape[1])])
    # select all events of this cluster
    event_idxs = np.argwhere(events_color_assignments==ecolor).flatten()
    for event_idx in event_idxs:
        # take start and end of the event
        estart = events[event_idx]['start']
        eend = events[event_idx]['end']
        row_mask[ estart:eend ] = True
    # remove cores whose firing rate outside its cluster events is higher than inside
    # because it means they are firing unspecifically
    for coids in clusters_cores_by_color[ecolor]:
        if type(coids)!=type([]): # for single element list retrieved as element
            coids = [coids]
        for coid in coids:
            coidx = ophys_cell_ids.index(coid)
            event_meanrate = np.mean(np_cells_firing_rate[coidx][row_mask])
            outside_meanrate = np.mean(np_cells_firing_rate[coidx])
            meanratediff = event_meanrate-outside_meanrate
            if meanratediff < 0.:
                if caidx in clusters_cores:
                    if coid in clusters_cores[caidx]:
                        clusters_cores[caidx].remove(coid)

np.save(exp_path+"/results/clusters_cores%s.npy"%scan_id, clusters_cores)


Gathering all cores ids and indexes.

In [ ]:
print("    gathering cores from all clusters")
core_indexes = []
other_indexes = []
for dyn_core in clusters_cores:
    core_indexes.extend( [ophys_cell_ids.index(strid) for strid in dyn_core] )
core_indexes = np.unique(core_indexes)
print("    # cores:",len(core_indexes))
other_indexes = [i for i in range(len(ophys_cell_ids)) if i not in core_indexes]
print("    # non-cores:",len(other_indexes))
# details
cxc = []
oxc = []
for ccl in clusters_cores_by_color.values():
    cxc.append(len(ccl))
    ool = [1 for i in ophys_cell_ids if i not in ccl]
    oxc.append(len(ool))
print("    cores per cluster: {:1.2f}±{:1.2f} (min {:d}, max {:d})".format(np.mean(cxc),np.std(cxc),np.min(cxc),np.max(cxc)) )
print("    others per cluster: {:1.2f}±{:1.2f} (min {:d}, max {:d})".format(np.mean(oxc),np.std(oxc),np.min(oxc),np.max(oxc)) )